# Post Process Report

In [ ]:
import re
import csv
import pandas as pd

In [ ]:
read_filepath = "outputs/ScansReport.csv"

In [ ]:
data = []

In [ ]:
with open(read_filepath, 'r', encoding="UTF-8") as f:
    reader = csv.reader(f, delimiter=',')
    data = list(reader)

Exclude Column

VA

```
See Also
Plugin Output
Asset UUID
NetBios
MAC Address
CVSS Temporal Score
CVSS3 Temporal Score
CVSS3 Temporal Vector

```

CIS
```
See Also
Plugin Output
Asset UUID
NetBios
MAC Address
Plugin Family
```

In [ ]:
hardening = [obj for obj in data if re.findall(r"(windows|unix)\scompliance\schecks", obj[7].lower())]

In [ ]:
def extractText(data):
    extracted_data = {}
    locate_index = []

    for i in range(0, len(data)):
        text = data[i]

        ifFindStatus = re.search(r'\[(passed|failed|warning|error)\]', text.lower())
        ifFindColumns = re.search(r'(rationale|solution|see also|reference|policy value|actual value)\:', text.lower())

        if ifFindStatus:
            locate_index.append(i)
        elif ifFindColumns:
            locate_index.append(i)


    title = data[:locate_index[1]]

    for text in title:
        text = text.strip('"')
        ifFindStatus = re.search(r'\[(passed|failed|warning|error)\]', text.lower())
        if ifFindStatus:
            title_name = text[0:ifFindStatus.start()-1]
            title_status = text[ifFindStatus.start()+1: ifFindStatus.end()-1]

            title_name = title_name.strip(":").strip(" ").split(' - ')
            extracted_data["Template"] = title_name[0]
            extracted_data["Rule Name"] = title_name[1] if len(title_name)>=2 else ""
            extracted_data["Module"] = title_name[2] if len(title_name)==3 else ""
            
#             extracted_data["title"] = title_name.strip(":").strip(" ").split(' - ')
            extracted_data["Status"] = title_status
        else:
            extracted_data["Description"] = text

    for i in range(1, len(locate_index)-1):
        extracted_data[data[locate_index[i]].strip(":")] = "\n".join(data[locate_index[i]+1:locate_index[i+1]])
    extracted_data[data[locate_index[-1]].strip(":")] = "\n".join(data[locate_index[-1]+1:])

    return extracted_data

In [ ]:
hardening_data = [h[9] for h in hardening]

results = []
for hardening_d in hardening_data:
    data = hardening_d
    data = [re.sub(r"^\s+", "", line) for line in data.split("\n") if line != ""]
    results.append(extractText(data))

In [ ]:
columns = ['Template', 'Rule Name', 'Module', 'Status', 'Description', 'Rationale', 'Solution', 'Reference', 'Policy Value', 'Actual Value']
results_df = pd.DataFrame(results, columns=columns)
results_df = results_df.fillna('N/A')
results_df

In [ ]:
filepath = "outputs/postReport.csv"
results_df.to_csv(filepath, index=False, encoding='utf-8')